In [1]:
import pandas as pd
import cv2
import csv
import random
import keras

from keras.models import Sequential, load_model, Model
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Dropout, Lambda, SpatialDropout2D
from keras.layers import Conv2D, Cropping2D, LSTM, Input, GRU
from keras.layers import MaxPooling2D,AveragePooling2D, concatenate
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from keras.backend import squeeze

Using TensorFlow backend.


In [2]:
centers_train = []
lefts_train = []
rights_train = []
centers_valid = []
lefts_valid = []
rights_valid = []

In [3]:
range_speed = (32 - 3)/2.
center_speed = (32 + 3)/2.

In [4]:
df = pd.read_csv("./Data1/driving_log.csv")
for i in df.index:
    centers_train.append(['Data1/IMG/' + df['center'][i].split('\\')[-1], df['steering'][i], \
                  (df['speed'][i] - center_speed)/range_speed])
    lefts_train.append(['Data1/IMG/' + df['left'][i].split('\\')[-1], df['steering'][i] + 0.18, \
                 (df['speed'][i] - center_speed)/range_speed]) 
    rights_train.append(['Data1/IMG/' + df['right'][i].split('\\')[-1], df['steering'][i]- 0.18, \
                 (df['speed'][i] - center_speed)/range_speed])

df = pd.read_csv("./Data2/driving_log.csv")
for i in df.index:
    centers_valid.append(['Data2/IMG/' + df['center'][i].split('\\')[-1], df['steering'][i], \
                  (df['speed'][i] - center_speed)/range_speed])
    lefts_valid.append(['Data2/IMG/' + df['left'][i].split('\\')[-1], df['steering'][i] + 0.18, \
                 (df['speed'][i] - center_speed)/range_speed]) 
    rights_valid.append(['Data2/IMG/' + df['right'][i].split('\\')[-1], df['steering'][i]- 0.18, \
                 (df['speed'][i] - center_speed)/range_speed])

In [5]:
centers_train[:5]

[['Data1/IMG/center_2018_12_13_19_32_40_877.jpg',
  -0.08823529,
  -0.4365572413793104],
 ['Data1/IMG/center_2018_12_13_19_32_40_949.jpg',
  -0.08823529,
  -0.43972344827586207],
 ['Data1/IMG/center_2018_12_13_19_32_41_021.jpg',
  -0.08823529,
  -0.43503448275862067],
 ['Data1/IMG/center_2018_12_13_19_32_41_091.jpg',
  -0.08823529,
  -0.4385344827586207],
 ['Data1/IMG/center_2018_12_13_19_32_41_161.jpg',
  -0.08823529,
  -0.44471103448275867]]

In [6]:
centers_train[-5:]

[['Data1/IMG/center_2018_12_13_19_38_55_337.jpg',
  -0.05882353,
  -0.010673793103448464],
 ['Data1/IMG/center_2018_12_13_19_38_55_409.jpg',
  -0.05882353,
  -0.020114482758620707],
 ['Data1/IMG/center_2018_12_13_19_38_55_482.jpg',
  -0.05882353,
  -0.029479999999999996],
 ['Data1/IMG/center_2018_12_13_19_38_55_564.jpg', 0.0, -0.03875586206896546],
 ['Data1/IMG/center_2018_12_13_19_38_56_228.jpg', 0.0, -0.2813351724137932]]

In [7]:
samples_train =[]
samples_valid = []

In [8]:
len(centers_train)

5196

In [9]:
def preprocess_data(datas, flag, type_input):
    if type_input == "Train":
        len_data = len(datas)
        
        for offset in range(len_data-5):
            batch_samples = datas[offset:offset+5]
            temp = []
            for data in batch_samples:
                temp.append([data[0], data[1], data[2], flag])
            samples_train.append(temp)
            
    if type_input == "Valid":
        len_data = len(datas)
        
        for offset in range(len_data-5):
            batch_samples = datas[offset:offset+5]
            temp = []
            for data in batch_samples:
                temp.append([data[0], data[1], data[2], flag])
            samples_valid.append(temp)
        

In [10]:
preprocess_data(centers_train, 'origin', 'Train')


In [11]:
preprocess_data(lefts_train, 'origin', 'Train')
preprocess_data(rights_train, 'origin', 'Train')

preprocess_data(centers_train, 'flip', 'Train')
preprocess_data(lefts_train, 'flip', 'Train')
preprocess_data(rights_train, 'flip', 'Train')

preprocess_data(lefts_valid, 'origin', 'Valid')
preprocess_data(rights_valid, 'origin', 'Valid')
preprocess_data(centers_valid, 'origin', 'Valid')

In [12]:
print (len(samples_train))
print (len(samples_valid))

31146
4476


In [13]:
print(samples_train[0])
print ("---------------------")
print(samples_train[0][1])

[['Data1/IMG/center_2018_12_13_19_32_40_877.jpg', -0.08823529, -0.4365572413793104, 'origin'], ['Data1/IMG/center_2018_12_13_19_32_40_949.jpg', -0.08823529, -0.43972344827586207, 'origin'], ['Data1/IMG/center_2018_12_13_19_32_41_021.jpg', -0.08823529, -0.43503448275862067, 'origin'], ['Data1/IMG/center_2018_12_13_19_32_41_091.jpg', -0.08823529, -0.4385344827586207, 'origin'], ['Data1/IMG/center_2018_12_13_19_32_41_161.jpg', -0.08823529, -0.44471103448275867, 'origin']]
---------------------
['Data1/IMG/center_2018_12_13_19_32_40_949.jpg', -0.08823529, -0.43972344827586207, 'origin']


In [14]:
def mini_model():
    img_input = Input(shape=(160, 320, 3))
    crop_layer = Cropping2D(cropping=((60, 0), (0,0)))(img_input)
    lambda_layer = Lambda(lambda x: x/127.5 - 1.)(crop_layer)
    
    conv1 = Conv2D(16, (5,5), padding='same', activation='relu')(lambda_layer)
    pool_layer1 = AveragePooling2D(pool_size=(2, 5))(conv1)
    
    conv_layer1 = Conv2D(32, (5, 5), activation='relu')(pool_layer1)
    pool_layer2 = MaxPooling2D()(conv_layer1)
    
    conv_layer2 = Conv2D(64, (5, 5), activation='relu')(pool_layer2)
    pool_layer3 = MaxPooling2D()(conv_layer2)
    
    conv_layer3 = Conv2D(72, (3, 3), activation='relu')(pool_layer3)
    conv_layer4 = Conv2D(96, (3, 3), activation='relu')(conv_layer3)
    conv_layer5 = Conv2D(112, (3, 3), activation='relu')(conv_layer4)
    conv_layer6 = Conv2D(128, (3, 3), activation='relu')(conv_layer5)
    flatten = Flatten()(conv_layer6)
    dense1 = Dense(300, activation='relu')(flatten)
    dense2 = Dense(100, activation='relu')(dense1)
    return Model(inputs=[img_input], outputs = [dense2])
    
def model_control():
    share_conv = mini_model()
#     share_conv.summary()

    input1 = Input(shape=(160, 320, 3))
    input2 = Input(shape=(160, 320, 3))
    input3 = Input(shape=(160, 320, 3))
    input4 = Input(shape=(160, 320, 3))
    input5 = Input(shape=(160, 320, 3))

    conv1 = share_conv(input1)
    conv2 = share_conv(input2)
    conv3 = share_conv(input3)
    conv4 = share_conv(input4)
    conv5 = share_conv(input5)
    

    out = concatenate(inputs=[conv1,conv2,conv3,conv4,conv5], axis=-1)
    out = keras.layers.Reshape(target_shape=(5, 100))(out)
    out = GRU((100),activation="relu",return_sequences=True)(out)
    out = GRU((100),activation="relu",return_sequences=True)(out)
    out = GRU((100),activation="relu",return_sequences=True)(out)
#     out = LSTM((100),activation="relu",return_sequences=True)(out)
    flat = Flatten()(out)
    
    fc1 = Dense(100,activation="relu")(flat)
    drop1 = Dropout(0.4)(fc1)
    fc2 = Dense(50,activation="relu")(drop1)
    fc3 = Dense(1)(fc2)
    
    fc4 = Dense(100,activation="relu")(flat)
    drop2 = Dropout(0.4)(fc4)
    fc5 = Dense(50,activation="relu")(drop2)
    fc6 = Dense(1)(fc5)
    return Model(inputs=[input1, input2, input3, input4, input5], outputs = [fc3,fc6])

In [15]:
model = model_control()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 160, 320, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 160, 320, 3)  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 160, 320, 3)  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 160, 320, 3)  0                                            
__________________________________________________________________________________________________
input_6 (I

In [19]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: 
        samples = shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images_1 = []
            images_2 = []
            images_3 = []
            images_4 = []
            images_5 = []
            
            angles = []
            speeds = []
            for batch_sample in batch_samples:
#                 print (batch_sample)
                try:
                    img1 = cv2.imread(batch_sample[0][0])
                    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)

                    img2 = cv2.imread(batch_sample[1][0])
                    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)

                    img3 = cv2.imread(batch_sample[2][0])
                    img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2HSV)

                    img4 = cv2.imread(batch_sample[3][0])
                    img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2HSV)

                    img5 = cv2.imread(batch_sample[4][0])
                    img5 = cv2.cvtColor(img5, cv2.COLOR_BGR2HSV)

                    angle = batch_sample[0][1]
                    speed = batch_sample[0][2]
                    if batch_sample[0][3] == 'flip':
                        img1 = cv2.flip(img1,1)
                        img2 = cv2.flip(img2,1)
                        img3 = cv2.flip(img3,1)
                        img4 = cv2.flip(img4,1)
                        img5 = cv2.flip(img5,1)
                        angle = -angle

                    images_1.append(img1)
                    images_2.append(img2)
                    images_3.append(img3)
                    images_4.append(img4)
                    images_5.append(img5)

                    angles.append([angle])
                    speeds.append([speed])
                except:
                    print (batch_sample)
            
            images_1 = np.array(images_1)
            images_2 = np.array(images_2)
            images_3 = np.array(images_3)
            images_4 = np.array(images_4)
            images_5 = np.array(images_5)
            
            steer_train = np.array(angles)
            speed_train = np.array(speeds)
            yield [images_1,images_2,images_3,images_4,images_5], [steer_train,speed_train]

In [20]:
print (len(samples_train))
print (len(samples_valid))

31146
4476


In [21]:
train_generator = generator(samples_train)
validation_generator = generator(samples_valid)

In [22]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0)
model.compile(loss='mse', optimizer=adam)

In [23]:
model.fit_generator(train_generator,
        steps_per_epoch=len(samples_train)//32,
        validation_data=validation_generator,
        validation_steps=len(samples_valid)//32,
        epochs=20)

Epoch 1/20
973/973 [==============================] - 627s 644ms/step - loss: 0.1003 - dense_5_loss: 0.0309 - dense_8_loss: 0.0693 - val_loss: 0.1718 - val_dense_5_loss: 0.0286 - val_dense_8_loss: 0.1433
Epoch 2/20
973/973 [==============================] - 466s 479ms/step - loss: 0.0757 - dense_5_loss: 0.0169 - dense_8_loss: 0.0588 - val_loss: 0.1069 - val_dense_5_loss: 0.0085 - val_dense_8_loss: 0.0984
Epoch 3/20
973/973 [==============================] - 396s 407ms/step - loss: 0.0478 - dense_5_loss: 0.0054 - dense_8_loss: 0.0424 - val_loss: 0.1486 - val_dense_5_loss: 0.0060 - val_dense_8_loss: 0.1425
Epoch 4/20
973/973 [==============================] - 606s 623ms/step - loss: 0.0388 - dense_5_loss: 0.0036 - dense_8_loss: 0.0352 - val_loss: 0.1453 - val_dense_5_loss: 0.0067 - val_dense_8_loss: 0.1387
Epoch 5/20
973/973 [==============================] - 931s 957ms/step - loss: 0.0353 - dense_5_loss: 0.0031 - dense_8_loss: 0.0322 - val_loss: 0.0865 - val_dense_5_loss: 0.0045 - val_d

KeyboardInterrupt: 

In [21]:
model.save("GRU.h5")

In [ ]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        samples = shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            speeds = []
            for batch_sample in batch_samples:
                center_image = cv2.imread(batch_sample[0])
                center_image = cv2.cvtColor(center_image, cv2.COLOR_BGR2HSV)
                center_angle = batch_sample[1]
                speed = batch_sample[2]
                if batch_sample[3] == 'flip':
                    center_image = cv2.flip(center_image,1)
                    center_angle = -batch_sample[1]
                images.append(center_image)
                angles.append([center_angle])
                speeds.append([speed])
            
            X_train = np.array(images)
            steer_train = np.array(angles)
            speed_train = np.array(speeds)
            yield [X_train], [steer_train,speed_train]

In [ ]:
def model_control():
    img_input = Input(shape=(160, 320, 3))
    crop_layer = Cropping2D(cropping=((60, 0), (0, 0)))(img_input)
    lambda_layer = Lambda(lambda x: x/127.5 - 1.)(crop_layer)
    
    conv1 = Conv2D(16, (5,5), padding='same', activation='relu')(lambda_layer)
    pool_layer1 = AveragePooling2D(pool_size=(2, 5))(conv1)
    
    conv_layer1 = Conv2D(32, (5, 5), activation='relu')(pool_layer1)
    pool_layer2 = MaxPooling2D()(conv_layer1)
    
    conv_layer2 = Conv2D(64, (5, 5), activation='relu')(pool_layer2)
    pool_layer3 = MaxPooling2D()(conv_layer2)
    
    conv_layer3 = Conv2D(72, (3, 3), activation='relu')(pool_layer3)
    conv_layer4 = Conv2D(96, (3, 3), activation='relu')(conv_layer3)
    conv_layer5 = Conv2D(112, (3, 3), activation='relu')(conv_layer4)
    conv_layer6 = Conv2D(128, (3, 3), activation='relu')(conv_layer5)
    
    flatten = Flatten()(conv_layer6)
    
    dense1 = Dense(300, activation='relu')(flatten)
    drop = Dropout(0.5)(dense1)
    dense2 = Dense(100, activation='relu')(drop)
    dense3 = Dense(50, activation='relu')(dense2)
    dense4 = Dense(1)(dense3)
    
    dense5 = Dense(100, activation='relu')(drop)
    dense6 = Dense(50, activation='relu')(dense5)
    dense7 = Dense(1)(dense6)
    
    return Model(inputs=[img_input], outputs = [dense4,dense7])

In [ ]:
model = model_control()
model.summary()

In [ ]:
samples = shuffle(samples)
train_samples, validation_samples = train_test_split(samples, test_size=0.2)


In [ ]:
train_generator = generator(train_samples)
validation_generator = generator(validation_samples)


In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.01, decay=0.0)
model.compile(loss='mse', optimizer=adam)

In [ ]:
model.fit_generator(train_generator,
        steps_per_epoch=len(train_samples)//32,
        validation_data=validation_generator,
        validation_steps=len(validation_samples)//32,
        epochs=10)

In [ ]:
model.save('./model_diff1.h5')
# model.save_weights('./weight.h5')

In [ ]:
img = cv2.imread('Data/IMG/center_2018_10_18_10_40_28_988.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

In [ ]:
a = model.predict(img[None, ...])

In [ ]:
a